In [14]:
# import some library
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pickle
import matplotlib.pyplot as plt
import itertools
import time
from keras.models import load_model

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GridSearchCV

from keras import models
from keras import layers
from keras.layers.advanced_activations import LeakyReLU
from keras.utils import np_utils
from keras.layers import Dense, Dropout, Activation, Flatten, Input, Dense, GaussianNoise, GaussianDropout
from keras.models import Sequential, Model
from keras.regularizers import l2, l1
from keras.optimizers import Adam, SGD, Nadam, Adamax
from keras.callbacks import LearningRateScheduler
from keras.metrics import categorical_accuracy
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping
from keras.wrappers.scikit_learn import KerasClassifier
from  keras.regularizers import l1, l2
from keras import backend as K
from keras.utils.generic_utils import get_custom_objects

import matplotlib.pyplot as plt

%matplotlib inline

import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto(
    gpu_options=tf.GPUOptions(
        visible_device_list="1", 
        allow_growth=True,
#         per_process_gpu_memory_fraction=0.5
    )
)
set_session(tf.Session(config=config))

In [15]:
# read data
df = pd.read_csv('../data/feature_importance_pos.csv', index_col=0)

# divide objective and target
objective = df.Subclass
le = preprocessing.LabelEncoder()
objective = le.fit_transform(objective)
features = df.drop('Subclass', axis=1)

# train test split
random_state=np.random.seed(42)
X_train, X_test, y_train, y_test = train_test_split(
    features, 
    objective,
    test_size=0.2
)

In [16]:
# transform  for keras's target label
y_train_for_keras = np_utils.to_categorical(y_train)
y_test_for_keras = np_utils.to_categorical(y_test)

In [19]:
# gridsearch
for i in [128, 256, 512]:  # layer
    for p in ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']:  # optimizers
        for m in ['softmax','softplus','softsign','relu','tanh','sigmoid','hard_sigmoid','linear']:  # activations
            tf.set_random_seed(42)

            # make keras model
            start = time.time()
            inputs = Input(shape=(X_train.shape[1],))
            x = Dense(i, activation=m)(inputs)
            x = BatchNormalization()(x)
            x = Dropout(0.5)(x)
            x = Dense(i, activation=m)(x)
            x = BatchNormalization()(x)
            x = Dropout(0.5)(x)
            x = Dense(i, activation=m)(x)
            x = BatchNormalization()(x)
            x = Dropout(0.5)(x)
            x = Dense(i, activation=m)(x)
            x = BatchNormalization()(x)
            x = Dropout(0.5)(x)
            predictions = Dense(len(df['Subclass'].value_counts()), activation='softmax')(x)
            model = Model(inputs=inputs, outputs=predictions)

            # compile
            model.compile(
                loss='categorical_crossentropy',
                optimizer=p,
                metrics=['accuracy']
            )

            epochs = 100
            batch_size = 1000
            es = EarlyStopping(monitor='val_loss', patience=20)

            history = model.fit(
                X_train,
                y_train_for_keras,
                epochs=epochs,
                batch_size=batch_size,
                validation_data=(X_test, y_test_for_keras),
                verbose=0,
                callbacks=[
                    es,
                ]
            )

            print(i, p, model.evaluate(X_test, y_test_for_keras, verbose=0)[1])

            if model.evaluate(X_test, y_test_for_keras, verbose=0)[1] > load_model('../model/Keras_fs_pos.h5').evaluate(X_test, y_test_for_keras, verbose=0)[1]:
                model.save('../model/Keras_fs_pos.h5')

128 SGD 0.3244047619047619
128 RMSprop 0.7083333333333334
128 Adagrad 0.6071428571428571
128 Adadelta 0.5505952380952381
128 Adam 0.5863095238095238
128 Adamax 0.4375
128 Nadam 0.6577380952380952
256 SGD 0.4166666666666667
256 RMSprop 0.6696428571428571
256 Adagrad 0.6904761904761905
256 Adadelta 0.6309523809523809
256 Adam 0.6726190476190477
256 Adamax 0.5297619047619048
256 Nadam 0.6339285714285714
512 SGD 0.39880952380952384
512 RMSprop 0.6011904761904762
512 Adagrad 0.6636904761904762
512 Adadelta 0.6488095238095238
512 Adam 0.6815476190476191
512 Adamax 0.6607142857142857
512 Nadam 0.6964285714285714


In [10]:
# tf.set_random_seed(42)

# # make keras model
# start = time.time()
# inputs = Input(shape=(X_train.shape[1],))

# x = Dense(512, activation='relu')(inputs)
# x = BatchNormalization()(x)
# x = Dropout(0.5)(x)

# x = Dense(512, activation='relu')(x)
# x = BatchNormalization()(x)
# x = Dropout(0.5)(x)

# x = Dense(512, activation='relu')(x)
# x = BatchNormalization()(x)
# x = Dropout(0.5)(x)

# x = Dense(512, activation='relu')(x)
# x = BatchNormalization()(x)
# x = Dropout(0.5)(x)

# predictions = Dense(
#     len(df['Subclass'].value_counts()), activation='softmax')(x)
# model = Model(inputs=inputs, outputs=predictions)

# # compile
# model.compile(
#     loss='categorical_crossentropy',
#     optimizer=Adamax(),
#     metrics=['accuracy']
# )

# epochs = 100
# batch_size = 1000
# es = EarlyStopping(monitor='val_loss', patience=20)

# history = model.fit(
#     X_train,
#     y_train_for_keras,
#     epochs=epochs,
#     batch_size=batch_size,
#     validation_data=(X_test, y_test_for_keras),
#     verbose=1,
#     callbacks=[
#         es,
#     ]
# )

# model.save('../model/Keras_fs_pos.h5')

Train on 1340 samples, validate on 336 samples
Epoch 1/100
1340/1340 [==============================] - 5s 4ms/step - loss: 1.9250 - acc: 0.2440 - val_loss: 1.4684 - val_acc: 0.2887
Epoch 2/100
1340/1340 [==============================] - 0s 100us/step - loss: 1.7934 - acc: 0.2560 - val_loss: 1.4713 - val_acc: 0.2798
Epoch 3/100
1340/1340 [==============================] - 0s 96us/step - loss: 1.6911 - acc: 0.2970 - val_loss: 1.3848 - val_acc: 0.3036
Epoch 4/100
1340/1340 [==============================] - 0s 90us/step - loss: 1.5790 - acc: 0.2903 - val_loss: 1.3398 - val_acc: 0.3036
Epoch 5/100
1340/1340 [==============================] - 0s 89us/step - loss: 1.5932 - acc: 0.3112 - val_loss: 1.2824 - val_acc: 0.3333
Epoch 6/100
1340/1340 [==============================] - 0s 94us/step - loss: 1.5472 - acc: 0.3231 - val_loss: 1.2621 - val_acc: 0.3363
Epoch 7/100
1340/1340 [==============================] - 0s 93us/step - loss: 1.4306 - acc: 0.3433 - val_loss: 1.2563 - val_acc: 0.3393
E

1340/1340 [==============================] - 0s 96us/step - loss: 0.9442 - acc: 0.5828 - val_loss: 0.9573 - val_acc: 0.5804
Epoch 61/100
1340/1340 [==============================] - 0s 96us/step - loss: 0.9051 - acc: 0.5955 - val_loss: 0.9511 - val_acc: 0.5923
Epoch 62/100
1340/1340 [==============================] - 0s 97us/step - loss: 0.8962 - acc: 0.6030 - val_loss: 0.9444 - val_acc: 0.6071
Epoch 63/100
1340/1340 [==============================] - 0s 97us/step - loss: 0.8827 - acc: 0.6045 - val_loss: 0.9391 - val_acc: 0.6131
Epoch 64/100
1340/1340 [==============================] - 0s 93us/step - loss: 0.8771 - acc: 0.6127 - val_loss: 0.9348 - val_acc: 0.6042
Epoch 65/100
1340/1340 [==============================] - 0s 95us/step - loss: 0.8820 - acc: 0.6097 - val_loss: 0.9313 - val_acc: 0.6071
Epoch 66/100
1340/1340 [==============================] - 0s 112us/step - loss: 0.8718 - acc: 0.6090 - val_loss: 0.9275 - val_acc: 0.6101
Epoch 67/100
1340/1340 [=============================

0.7083333333333334